# Pre-Processing

In [2]:
import requests as req
# # read in both train and test data
url_train = 'https://raw.githubusercontent.com/vickybwu/Myfiles/main/train.txt'
res = req.get(url_train)
train_data = res.text

url_test = 'https://raw.githubusercontent.com/vickybwu/Myfiles/main/test.txt'
ress = req.get(url_test)
test_data = ress.text

In [3]:
# split the corpus into sentences 
train_data_sentenced = train_data.split('\n')[: -1]
test_data_sentenced = test_data.split('\n')[: -1]

In [4]:
# pad the training data
begin_token = '<s> '
end_token = ' </s>'
train_data_padded = list(map(lambda x: begin_token + x + end_token, train_data_sentenced))

# pad the testing data 
test_data_padded = list(map(lambda x: begin_token + x + end_token, test_data_sentenced))


In [5]:
from string import punctuation
## split the padded training coupus into words
trained_words = []
for sen in train_data_padded:
  words = sen.lower().split(' ')
  trained_words += words
  # for word in words:
  #   if word == '<s>' or word == '</s>':
  #     trained_words.append(word)
  #   else:
  #     word = word.strip(punctuation)
  #     if word != '':
  #       trained_words.append(word)

## split the padded testing coupus into words
test_words = []
for sen in test_data_padded:
  words = sen.lower().split(' ')
  test_words += words
  # for word in words:
  #   if word == '<s>' or word == '</s>':
  #     test_words.append(word)
  #   else:
  #     word = word.strip(punctuation)
  #     if word != '':
  #       test_words.append(word)

In [6]:
unique_words = {}
for word in trained_words:
  if word not in unique_words.keys():
    unique_words[word] = 1
  else:
    unique_words[word] += 1

unk_words = [key for key,value in unique_words.items() if value == 1]

### Question 1

In [397]:
# question 1
total_word_type_count = len(list(unique_words.keys())) # including start and end symbol
unk_word_count = len(unk_words)
word_type_count = total_word_type_count - unk_word_count -1 + 1 # exluding start symbol and including <unk>
print("The total word types in the training corpus is:", word_type_count)

The total word types in the training corpus is: 41738


### Question 2

In [7]:
total_token_count = len(trained_words)
start_symbol_count = unique_words['<s>']
total_token_without_start_count = total_token_count - start_symbol_count
print("The total token count excluding the start symbol is:",total_token_without_start_count)

The total token count excluding the start symbol is: 2468210


### Question 3

In [8]:
# before mapping words to <unk> in train ot test data
from collections import Counter
frequency_test = Counter(test_words)
# print(len(frequency_test))
start_token_count = frequency_test['<s>']
#print(start_token_count)
token_count_ex_start_test = len(test_words) - start_token_count
total_word_type = len(list(frequency_test.keys()))
word_type_ex_start_test = total_word_type-1

# word types not appeared in training data
train_word_type = list(unique_words.keys())
test_word_type = list(frequency_test.keys())
test_word_not_in_training = list(set(test_word_type) - set(train_word_type))


a1 = len(test_word_not_in_training)/word_type_ex_start_test
a2 = sum(frequency_test[word] for word in test_word_not_in_training)/token_count_ex_start_test
print("{:.2%}".format(a1), "of word types in the testing data is not in the training data (excluding the start symbol)")
print("{:.2%}".format(a2), "of the tokens in the testing data is not in the training data (excluding the start symbol)")

3.61% of word types in the testing data is not in the training data (excluding the start symbol)
1.66% of the tokens in the testing data is not in the training data (excluding the start symbol)


### Question 4


In [15]:
# replace words appeared once in training with <unk>
trained_words_unk = []
for word in trained_words:
    if word in unk_words:
        trained_words_unk.append('<unk>')
    else:
        trained_words_unk.append(word)    

In [16]:
# replace test words not in training to unk
# trained_words_unk = list(map(lambda x: '<unk>' if x in unk_words else x, trained_words))
test_word_not_in_training_2 = list(set(test_word_type) - set(trained_words_unk))
test_words_unk = list(map(lambda x: '<unk>' if x in test_word_not_in_training_2 else x, test_words))

In [17]:
# create bigram dictionary for the training data
bigram_train = {}
for i in range(len(trained_words_unk)-1):
    if trained_words_unk[i] != '<s>' and trained_words_unk[i+1] != '<s>':
        bigram = (trained_words_unk[i], trained_words_unk[i+1])
        if bigram not in bigram_train.keys():
            bigram_train[bigram] = 1
        else: 
            bigram_train[bigram] += 1
# create bigram dictionary for the test data
bigram_test = {}
for i in range(len(test_words_unk)-1):
    if test_words_unk[i] != '<s>' and test_words_unk[i+1] != '<s>':
        bigram = (test_words_unk[i], test_words_unk[i+1])
        if bigram not in bigram_test.keys():
            bigram_test[bigram] = 1
        else:
            bigram_test[bigram] += 1

print("There are", len(bigram_train), "bigram types in the training data")
print("There are", sum(bigram_train.values()), "bigrams in the training data")
print("There are", len(bigram_test), "bigram types in the test data")
print("There are", sum(bigram_test.values()), "bigrams in the test data")

There are 731347 bigram types in the training data
There are 2368210 bigrams in the training data
There are 2300 bigram types in the test data
There are 2669 bigrams in the test data


In [18]:
# calculate percentage of bigram types in the test data not in the training data 
diff_bitypes = list(set(bigram_test.keys()) - set(bigram_train.keys()))
bitype_diff_count = len(diff_bitypes)
bitype_diff_perc = bitype_diff_count/len(bigram_test)
# calculate percentage of bigram tokens in the test data not in the training data 
bitoken_diff_perc = sum([bigram_test[gram] for gram in diff_bitypes])/sum(bigram_test.values())

print("There are", "{:.2%}".format(bitype_diff_perc), "percent of bigram types in the test data not in the training data")
print("There are", "{:.2%}".format(bitoken_diff_perc), "percent of bigram tokens in the test data not in the training data")

There are 25.87% percent of bigram types in the test data not in the training data
There are 22.37% percent of bigram tokens in the test data not in the training data


### Question 5

In [19]:
test_sentence = 'I look forward to hearing your reply.'
tokenized_test = test_sentence.strip(punctuation).lower().split(' ')
# Pad the sentence with begin and end symbols
tokenized_test = ['<s>'] + tokenized_test + ['</s>']
tokenized_test

['<s>', 'i', 'look', 'forward', 'to', 'hearing', 'your', 'reply', '</s>']

In [20]:
# Create a dictionary for every word type and its count for the training data after <unk> is implemented
train_word_type_dict = Counter(trained_words_unk)

In [21]:
# replace unigrams in the test sentence not in the training data with <unk>
for i in range(len(tokenized_test)):
    if tokenized_test[i] not in train_word_type_dict.keys():
        tokenized_test[i] = '<unk>'
print(tokenized_test)

['<s>', 'i', 'look', 'forward', 'to', 'hearing', 'your', 'reply', '</s>']


#### Unigram Model

In [22]:
import numpy as np
# calculate the log probability of the sentence using unigram model
def log_unigram_prob(train_word_type_dict, test):
    total_token_in_train = sum(train_word_type_dict.values())
    sentence_prob = 1
    for token in test:
        count_of_token = train_word_type_dict[token]
        prob_of_token = count_of_token/total_token_in_train
        #print("The probability of having token", token, "in the sentence is", prob_of_token)
        sentence_prob *= prob_of_token
    log_sen_prob = np.log2(sentence_prob)
    return log_sen_prob
unigram_probability = log_unigram_prob(train_word_type_dict, tokenized_test)
print("The log probability of the sentence under unigram model is:", unigram_probability)

The log probability of the sentence under unigram model is: -90.06992741329718


#### Bigram Model

In [23]:
#convert training text into a dictionary of bigrams 
bigram_train_2 = {}
for i in range(len(trained_words_unk)-1):
        bigram = (trained_words_unk[i], trained_words_unk[i+1])
        if bigram not in bigram_train_2.keys():
            bigram_train_2[bigram] = 1
        else: 
            bigram_train_2[bigram] += 1

#convert test sentence into a list of bigrams
bigram_test_2 = []
for i in range(len(tokenized_test)-1):
        bigram = (tokenized_test[i], tokenized_test[i+1])
        bigram_test_2.append(bigram)

In [24]:
# calculate the log probability of the sentence under the bigram model
def log_bigram_prob(train_bigram_dict, train_unigram_dict, test_bigram_list):
    sentence_prob = 1
    for bigram in test_bigram_list:
        if bigram not in train_bigram_dict.keys():
            print("The following paramter has 0 value:", "P", bigram)
            return 0
        else:
            prob_of_bigram = train_bigram_dict[bigram]/train_unigram_dict[bigram[0]]
            sentence_prob *= prob_of_bigram
    return np.log2(sentence_prob)

bigram_probability = log_bigram_prob(bigram_train_2, train_word_type_dict, bigram_test_2)
print("The log probability of the sentence under the bigram model is", bigram_probability)

The following paramter has 0 value: P ('hearing', 'your')
The log probability of the sentence under the bigram model is 0


#### Bigram Model with Add-one Smoothing

In [37]:
# calculate the log probability of the sentence under the bigram model with add one smoothing
def log_bigram_smoothed(train_bigram_dict, train_unigram_dict, test_bigram_list):
    sentence_prob = 1
    V = len(train_unigram_dict)
    for bigram in test_bigram_list:
        if bigram not in train_bigram_dict.keys():
            numerator = 0
        else:
            numerator = train_bigram_dict[bigram]
        prob_of_bigram = (numerator+1)/(train_unigram_dict[bigram[0]]+V)
        sentence_prob += np.log2(prob_of_bigram)
    return sentence_prob

log_bigram_smoothed_probability = log_bigram_smoothed(bigram_train_2, train_word_type_dict, bigram_test_2)
print("The log probability of the sentence under the bigram model with add one smoothing is", bigram_probability)

The log probability of the sentence under the bigram model with add one smoothing is 0


#### Katz Backoff + Discounting on Bigram Model

In [26]:
def discounted_backoff_bigram(discount_value, train_bigram_dict, train_unigram_dict, test_bigram_list):
    sentence_prob = 1
    missing_bigram_base_words = {}
    alpha = {}
    # first find all the missing bigram's base words in the test data
    for bigram in test_bigram_list:
        if bigram not in train_bigram_dict.keys():
            w0 = bigram[0]
            wi = bigram[1]
            if w0 not in missing_bigram_base_words.keys():
                missing_bigram_base_words[w0] = {wi}
            else:
                missing_bigram_base_words[w0].add(wi)
    print("Missing bigrams:", missing_bigram_base_words)
    # calcualte alpha for each base word 0
    for w0 in missing_bigram_base_words.keys():
        total_base_word_count = train_unigram_dict[w0]
        total_discounts = sum([discount_value for key in train_bigram_dict.keys() if key[0] == w0])
        alpha[w0] = total_discounts/total_base_word_count
    print('alpha:', alpha)
    # Now calculate the probabilities for each bigram
    for bigram in test_bigram_list:
        if bigram[0] not in missing_bigram_base_words.keys() and bigram in train_bigram_dict.keys():
            bigram_prob = train_bigram_dict[bigram]/train_unigram_dict[bigram[0]]
            sentence_prob *= bigram_prob
        if bigram[0] in missing_bigram_base_words.keys() and bigram in train_bigram_dict.keys():
            bigram_prob = (train_bigram_dict[bigram]-discount_value)/train_unigram_dict[bigram[0]]
            sentence_prob *= bigram_prob
        if bigram[0] in missing_bigram_base_words.keys() and bigram not in train_bigram_dict.keys():
            w0 = bigram[0]
            wi = bigram[1]
            unigram_prob_numerator = train_unigram_dict[wi]/sum(train_unigram_dict.values())
            unigram_prob_denominator = sum([train_unigram_dict[x] for x in missing_bigram_base_words[w0]])/sum(train_unigram_dict.values())
            bigram_prob = alpha[bigram[0]] * unigram_prob_numerator/unigram_prob_denominator
            sentence_prob *= bigram_prob

    return np.log2(sentence_prob)

discounting_bigram_probability = discounted_backoff_bigram(0.5, bigram_train_2, train_word_type_dict, bigram_test_2)
print("The log probability of the sentence under the bigram model with .5 discounting is", discounting_bigram_probability)


Missing bigrams: {'hearing': {'your'}, 'your': {'reply'}, 'reply': {'</s>'}}
alpha: {'hearing': 0.18660287081339713, 'your': 0.28142974527526704, 'reply': 0.23076923076923078}
The log probability of the sentence under the bigram model with .5 discounting is -40.46781489609892


### Question 6

#### Perplexity of the models

In [27]:
# Perplexity of the sentence under the unigram model
def pp_unigram(train_word_type_dict, test):
    total_token_in_train = sum(train_word_type_dict.values())
    wordtype_test_count = len(test)
    Sum = 0
    for token in test:
        count_of_token = train_word_type_dict[token]
        prob_of_token = count_of_token/total_token_in_train
        Sum += np.log2(prob_of_token)
    perplexity = 2**(-Sum/wordtype_test_count)
    return perplexity
unigram_perplecity = pp_unigram(train_word_type_dict, tokenized_test)
print("Perplexity under the unigram model is", unigram_perplecity)


Perplexity under the unigram model is 1029.5296845225253


In [28]:
# Perplexity of the sentence under the bigram model
def pp_bigram(train_bigram_dict, train_unigram_dict, test_bigram_list, bigram_test_count):
    Sum = 0
    for bigram in test_bigram_list:
        if bigram not in train_bigram_dict.keys():
            prob_of_bigram = 0
            log_prob_of_bigram = np.log2(prob_of_bigram)
            Sum += log_prob_of_bigram

        else:
            prob_of_bigram = train_bigram_dict[bigram]/train_unigram_dict[bigram[0]]
            log_prob_of_bigram = np.log2(prob_of_bigram)
            Sum += log_prob_of_bigram
    perplexity = 2**(-Sum/bigram_test_count)
    return perplexity

bigram_perplexity = pp_bigram(bigram_train_2, train_word_type_dict, bigram_test_2, len(bigram_test_2))
print("Perplexity under the bigram model is", bigram_perplexity)

Perplexity under the bigram model is inf


/var/folders/vc/6cpgd0gd3d53rklf_fz231y80000gn/T/ipykernel_74313/284501343.py:7: RuntimeWarning: divide by zero encountered in log2
  log_prob_of_bigram = np.log2(prob_of_bigram)


In [29]:
def pp_bigram_smoothed(train_bigram_dict, train_unigram_dict, test_bigram_list, bigram_test_count):
    Sum = 0
    V = len(train_unigram_dict)
    for bigram in test_bigram_list:
        numerator = 0 if bigram not in train_bigram_dict.keys() else train_bigram_dict[bigram]
        prob_of_bigram = (numerator+1)/(train_unigram_dict[bigram[0]]+V)
        log_prob_of_bigram = np.log2(prob_of_bigram)
        Sum += log_prob_of_bigram
    perplexity = 2**(-Sum/bigram_test_count)
    return perplexity

bigram_perplexity_smoothed = pp_bigram_smoothed(bigram_train_2, train_word_type_dict, bigram_test_2, len(bigram_test_2))
print("Perplexity under the bigram model with add one smoothing is", bigram_perplexity_smoothed)

Perplexity under the bigram model with add one smoothing is 4275.263017416363


#### Perplexity of the bigram model with discounting

In [30]:
def pp_bigram_dicounted(discount_value, train_bigram_dict, train_unigram_dict, test_bigram_list, bigram_test_count):
    missing_bigram_base_words = {}
    alpha = {}
    Sum = 0
    # first find all the missing bigram's base words in the test data
    for bigram in test_bigram_list:
        if bigram not in train_bigram_dict.keys():
            w0 = bigram[0]
            wi = bigram[1]
            if w0 not in missing_bigram_base_words.keys():
                missing_bigram_base_words[w0] = {wi}
            else:
                missing_bigram_base_words[w0].add(wi)
   #print("Missing bigrams:", missing_bigram_base_words)
    # calcualte alpha for each base word 0
    for w0 in missing_bigram_base_words.keys():
        total_base_word_count = train_unigram_dict[w0]
        total_discounts = sum([discount_value for key in train_bigram_dict.keys() if key[0] == w0])
        alpha[w0] = total_discounts/total_base_word_count
    #print('alpha:', alpha)
    # Now calculate the probabilities for each bigram
    for bigram in test_bigram_list:
        if bigram[0] not in missing_bigram_base_words.keys() and bigram in train_bigram_dict.keys():
            bigram_prob = train_bigram_dict[bigram]/train_unigram_dict[bigram[0]]
            Sum += np.log2(bigram_prob)
        if bigram[0] in missing_bigram_base_words.keys() and bigram in train_bigram_dict.keys():
            bigram_prob = (train_bigram_dict[bigram]-discount_value)/train_unigram_dict[bigram[0]]
            Sum += np.log2(bigram_prob)
        if bigram[0] in missing_bigram_base_words.keys() and bigram not in train_bigram_dict.keys():
            w0 = bigram[0]
            wi = bigram[1]
            unigram_prob_numerator = train_unigram_dict[wi]/sum(train_unigram_dict.values())
            unigram_prob_denominator = sum([train_unigram_dict[x] for x in missing_bigram_base_words[w0]])/sum(train_unigram_dict.values())
            bigram_prob = alpha[bigram[0]] * unigram_prob_numerator/unigram_prob_denominator
            Sum += np.log2(bigram_prob)
    perplexity = 2**(-Sum/(bigram_test_count))
    return perplexity

bigram_dicounted_perplexity = pp_bigram_dicounted(0.5, bigram_train_2, train_word_type_dict, bigram_test_2, len(bigram_test_2))
print("The perplexity of the bigram model with discounting is:",bigram_dicounted_perplexity)

The perplexity of the bigram model with discounting is: 33.323703974856386


### Question 7

In [31]:
# further ensure all unique words to the test corpus is converted to <unk>
for i in range(len(test_words_unk)):
    if test_words_unk[i] not in train_word_type_dict.keys():
        test_words_unk[i] = '<unk>'
    else:
        continue

In [32]:
# For the padded test corpus, spplit it into sentences, make sure eveything is lowercased and all unique words to the test data is <unk>
test_corpus_sentences = []
for sentence in test_data_padded:
    sentence = sentence.lower()
    new_sentence = []
    for unigram in sentence.split(' '):
        if unigram not in train_word_type_dict.keys():
            unigram = '<unk>'
            new_sentence.append(unigram)
        else:
            new_sentence.append(unigram)
    test_corpus_sentences.append(' '.join(new_sentence))

In [33]:
# All the varibales needed 
## Test corpus unigram dictionary
test_unigram_dict = {}
for token in test_words_unk:
    if token not in test_unigram_dict.keys():
        test_unigram_dict[token] = 1
    else:
        test_unigram_dict[token] += 1

## Test corpus bigram list
test_corpus_bigrams = []
for i in range(len(test_words_unk)-1):
    bigram = (test_words_unk[i], test_words_unk[i+1])
    test_corpus_bigrams.append(bigram)
    
## Test corpus bigram dictionary
test_corpus_bigrams_dict = {}
for i in range(len(test_words_unk)-1):
    bigram = (test_words_unk[i], test_words_unk[i+1])
    if bigram not in test_corpus_bigrams_dict.keys():
        test_corpus_bigrams_dict[bigram] = 1
    else:
        test_corpus_bigrams_dict[bigram] += 1


#### Perplexity of the test corpus under unigram

In [41]:
def pp_corpus_unigram(train_word_type_dict, test_courpus_sentences, test_unigram_dict):
    total_token_in_train = sum(train_word_type_dict.values())
    test_unigram_count = sum(test_unigram_dict.values())
    Sum = 0
    for sentence in test_courpus_sentences:
        sentence_prob = 0
        unigrams = sentence.split(' ')
        for unigram in unigrams:
            log_prob_of_unigram = np.log2(train_word_type_dict[unigram]/total_token_in_train)
            sentence_prob += log_prob_of_unigram
        Sum += sentence_prob
    perplexity = 2**(-Sum/test_unigram_count)
    return perplexity
unigram_perplecity = pp_corpus_unigram(train_word_type_dict, test_corpus_sentences, test_unigram_dict)
print("Perplexity of the test corpus under the unigram model is", unigram_perplecity)

Perplexity of the test corpus under the unigram model is 1000.2124621763647


#### Perplexity of the entire test corpus under bigram

In [38]:
def pp_bigram_corpus(train_bigram_dict, train_unigram_dict, test_courpus_sentences, bigram_test_count):
    Sum = 0
    for sentence in test_courpus_sentences:
        words = sentence.split(' ')
        sentence_bigram_list = [(words[i], words[i+1]) for i in range(len(words)-1)]
        sentence_prob = 0
        for bigram in sentence_bigram_list:
            if bigram not in train_bigram_dict.keys():
                prob_of_bigram = np.log2(0)
                sentence_prob += prob_of_bigram

            else:
                prob_of_bigram = np.log2(train_bigram_dict[bigram]/train_unigram_dict[bigram[0]])
                sentence_prob += prob_of_bigram
        Sum += sentence_prob
    perplexity = 2**(-Sum/bigram_test_count)
    return perplexity

test_perplexity_bigram = pp_bigram_corpus(bigram_train_2, train_word_type_dict, test_corpus_sentences, len(test_corpus_bigrams))
print("The perplexity of the test corpus under bigram model is:", test_perplexity_bigram)

The perplexity of the test corpus under bigram model is: inf


/var/folders/vc/6cpgd0gd3d53rklf_fz231y80000gn/T/ipykernel_74313/1200326676.py:9: RuntimeWarning: divide by zero encountered in log2
  prob_of_bigram = np.log2(0)


#### Under bigram model with Add-one smoothing

In [39]:
def pp_bigram_corpus_smoothed(train_bigram_dict, train_unigram_dict, test_courpus_sentences, bigram_test_count, unique_bigram_test_count):
    Sum = 0
    V = unique_bigram_test_count
    for sentence in test_courpus_sentences:
        words = sentence.split(' ')
        sentence_bigram_list = [(words[i], words[i+1]) for i in range(len(words)-1)]
        sentence_prob = 0
        for bigram in sentence_bigram_list:
            if bigram not in train_bigram_dict.keys():
                numerator = 1
                log_prob_of_bigram = np.log2(numerator/(train_unigram_dict[bigram[0]]+V))
                sentence_prob += log_prob_of_bigram

            else:
                log_prob_of_bigram = np.log2((train_bigram_dict[bigram]+1)/(train_unigram_dict[bigram[0]]+V))
                sentence_prob += log_prob_of_bigram
        Sum += sentence_prob
        #print(Sum)
    perplexity = 2**(-Sum/bigram_test_count)
    return perplexity
test_corpus_perplexity_bigram_smoothed = pp_bigram_corpus_smoothed(bigram_train_2, train_word_type_dict, test_corpus_sentences, len(test_corpus_bigrams), len(test_corpus_bigrams_dict))
print("The perplexity of the test corpus under bigram model with add one smoothing is:", test_corpus_perplexity_bigram_smoothed)

The perplexity of the test corpus under bigram model with add one smoothing is: 323.68678658337


#### Under bigram with discounting

In [40]:
def pp_bigram_dicounted_corpus(discount_value, train_bigram_dict, train_unigram_dict, test_bigram_list, test_data_padded, bigram_test_count):
    missing_bigram_base_words = {}
    alpha = {}
    Sum = 0
    # first find all the missing bigram's base words in the test data
    for bigram in test_bigram_list:
        if bigram not in train_bigram_dict.keys():
            w0 = bigram[0]
            wi = bigram[1]
            if w0 not in missing_bigram_base_words.keys():
                missing_bigram_base_words[w0] = {wi}
            else:
                missing_bigram_base_words[w0].add(wi)
    # print("Missing bigrams:", missing_bigram_base_words)
    # calcualte alpha for each base word 0
    for w0 in missing_bigram_base_words.keys():
        total_base_word_count = train_unigram_dict[w0]
        if total_base_word_count == 0:
            print(w0)
        total_discounts = sum([discount_value*1 for key in train_bigram_dict.keys() if key[0] == w0])
        alpha[w0] = total_discounts/total_base_word_count
    # print('alpha:', alpha)
    # Now calculate the probabilities for each bigram
    for sentence in test_data_padded:
        words = sentence.split(' ')
        sentence_bigram_list = [(words[i], words[i+1]) for i in range(len(words)-1)]
        sentence_prob = 0      
        for bigram in sentence_bigram_list:
            if bigram[0] not in missing_bigram_base_words.keys() and bigram in train_bigram_dict.keys():
                bigram_prob = np.log2(train_bigram_dict[bigram]/train_unigram_dict[bigram[0]])
                sentence_prob += bigram_prob
            if bigram[0] in missing_bigram_base_words.keys() and bigram in train_bigram_dict.keys():
                bigram_prob = np.log2((train_bigram_dict[bigram]-discount_value)/train_unigram_dict[bigram[0]])
                sentence_prob += bigram_prob
            if bigram[0] in missing_bigram_base_words.keys() and bigram not in train_bigram_dict.keys():
                w0 = bigram[0]
                wi = bigram[1]
                unigram_prob_numerator = train_unigram_dict[wi]/sum(train_unigram_dict.values())
                unigram_prob_denominator = sum([(train_unigram_dict[x]/sum(train_unigram_dict.values())) for x in missing_bigram_base_words[w0]])
                bigram_prob = np.log2(alpha[w0] * unigram_prob_numerator/unigram_prob_denominator)
                sentence_prob += bigram_prob
        Sum += sentence_prob
    perplexity = 2**(-Sum/bigram_test_count)
    return perplexity

perplexity_test_corpus_discounted = pp_bigram_dicounted_corpus(.5, bigram_train_2, train_word_type_dict, 
test_corpus_bigrams, test_corpus_sentences, len(test_corpus_bigrams))
print("Perplexity of the test corpus under bigram model with discounting is:",perplexity_test_corpus_discounted)


Perplexity of the test corpus under bigram model with discounting is: 43.55225431918881
